In [2]:
import os

import cv2
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [3]:
ingredients_path = "../data/nutrition5k_dataset_nosides/metadata/ingredients_metadata.csv"
dish_metadata_path = "../data/nutrition5k_dataset_nosides/metadata/dish_metadata_cafe2.csv"
dish_ids_path = "../data/nutrition5k_dataset_nosides/dish_ids/dish_ids_all.txt"
img_dir = "../data/nutrition5k_dataset_nosides/imagery/realsense_overhead"

In [4]:
ingredients_df = pd.read_csv(ingredients_path)
ingredients_df.iloc[12]

ingr          berries
id                 13
cal/g            0.57
fat(g)          0.003
carb(g)          0.14
protein(g)      0.007
Name: 12, dtype: object

In [5]:
# Read the file line by line into a single-column DataFrame
df = pd.read_csv(dish_metadata_path, header=None, names=['raw'], sep="|")

# Split the 'raw' column into multiple columns on the comma character
df_split = df['raw'].str.split(',', expand=True)

# The number of columns minus the 7 initial fields, divided by 7 fields per ingredient
df_split['num_ingrs'] = df_split.count(axis=1).subtract(6) // 7

df_split['num_ingrs'].head()
df_split['num_ingrs'].to_csv("test.csv", index=False)

In [6]:
# Define the maximum number of columns expected
# Assuming a dish can have maximum 50 ingredients
max_columns = 7 + (50 * 7)

# Creating column names
# ['dish_id', 'total_calories', ... , 'ingr_50_protein']
columns = ['field_'+str(i) for i in range(1, max_columns+1)]

# Load the CSV and read it into a DataFrame
df = pd.read_csv(dish_metadata_path, names=columns)

# Display the first few rows of the DataFrame
df.head()
df.to_csv("test2.csv", index=False)

/var/folders/18/mk89nqqd4t1738cgptywpmw00000gn/T/ipykernel_8924/2377898611.py:10: DtypeWarning: Columns (230,231,237,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dish_metadata_path, names=columns)


In [7]:
pd

<module 'pandas' from '/Users/maxburzer/Desktop/SnackTrack-Backend/.venv/lib/python3.11/site-packages/pandas/__init__.py'>

In [8]:
dish_metadata_df = pd.read_csv(dish_metadata_path, on_bad_lines="skip")
dish_metadata_df.head()

,dish_1561662216,300.794281,193.000000,12.387489,28.218290,18.633970,ingr_0000000508,soy sauce,3.398568,1.80124104,...,0.000424822,0.019754223,0.00424822,ingr_0000000513,millet,3.414345,4.06307055,0.03414345,0.809199765,0.119502075
0,dish_1562688426,137.569992,88.0,8.256000,5.190000,10.297000,ingr_0000000433,roasted potatoes,17.000000,23.970000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dish_1561662054,419.438782,292.0,23.838249,26.351543,25.910593,ingr_0000000312,pepper,0.300716,0.120286,...,0.0,0.0,0.0,ingr_0000000524,parsley,0.752816,0.279295,0.006023,0.04818,0.021832
2,dish_1562008979,382.936646,290.0,22.224644,10.173570,35.345387,ingr_0000000448,jalapenos,0.192961,0.055959,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dish_1560455030,20.590000,103.0,0.148000,4.625000,0.956000,ingr_0000000471,cherry tomatoes,55.000000,9.900000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dish_1558372433,74.360001,143.0,0.286000,0.429000,20.020000,ingr_0000000453,deprecated,143.000000,74.360000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dish_metadata_df.iat[0, 0]

'dish_1562688426'

In [10]:
dish_ids = pd.read_csv(dish_ids_path, header=None)
dish_ids.head()
print(dish_ids.loc[0])

0    dish_1550704750
Name: 0, dtype: object


In [11]:
def get_ingredient_ids(filename):
    # Load the data into a pandas DataFrame
    df = pd.read_csv(filename, header=None)

    ingredient_ids = []

    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        ids = []

        # Iterate over each ingredient in the row
        # We start from column 7 (0-indexed), and increment by 7 for each ingredient
        for i in range(6, len(row), 7):
            # Check if ingredient id is not missing
            if pd.notnull(row[i]):
                # Get the ingredient id and remove any leading zeros
                ids.append(row[i])
        ingredient_ids.append(ids)

    return ingredient_ids

In [12]:
get_ingredient_ids("test2.csv")

/var/folders/18/mk89nqqd4t1738cgptywpmw00000gn/T/ipykernel_8924/2470206169.py:3: DtypeWarning: Columns (1,2,3,4,5,8,9,10,11,12,15,16,17,18,19,22,23,24,25,26,29,30,31,32,33,36,37,38,39,40,43,44,45,46,47,50,51,52,53,54,57,58,59,60,61,64,65,66,67,68,71,72,73,74,75,78,79,80,81,82,85,86,87,88,89,92,93,94,95,96,99,100,101,102,103,106,107,108,109,110,113,114,115,116,117,120,121,122,123,124,127,128,129,130,131,134,135,136,137,138,141,142,143,144,145,148,149,150,151,152,155,156,157,158,159,162,163,164,165,166,169,170,171,172,173,176,177,178,179,180,183,184,185,186,187,190,191,192,193,194,197,198,199,200,201,204,205,206,207,208,211,212,213,214,215,218,219,220,221,222,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,30

[['field_7',
  'field_14',
  'field_21',
  'field_28',
  'field_35',
  'field_42',
  'field_49',
  'field_56',
  'field_63',
  'field_70',
  'field_77',
  'field_84',
  'field_91',
  'field_98',
  'field_105',
  'field_112',
  'field_119',
  'field_126',
  'field_133',
  'field_140',
  'field_147',
  'field_154',
  'field_161',
  'field_168',
  'field_175',
  'field_182',
  'field_189',
  'field_196',
  'field_203',
  'field_210',
  'field_217',
  'field_224',
  'field_231',
  'field_238',
  'field_245',
  'field_252',
  'field_259',
  'field_266',
  'field_273',
  'field_280',
  'field_287',
  'field_294',
  'field_301',
  'field_308',
  'field_315',
  'field_322',
  'field_329',
  'field_336',
  'field_343',
  'field_350',
  'field_357'],
 ['ingr_0000000508',
  'ingr_0000000122',
  'ingr_0000000026',
  'ingr_0000000524',
  'ingr_0000000094',
  'ingr_0000000023',
  'ingr_0000000189',
  'ingr_0000000054',
  'ingr_0000000029',
  'ingr_0000000328',
  'ingr_0000000291',
  'ingr_0000000520

In [13]:
df = pd.read_csv("test2.csv")
df[df.iloc[:, 0] == "dish_1550704750"]

/var/folders/18/mk89nqqd4t1738cgptywpmw00000gn/T/ipykernel_8924/63870995.py:1: DtypeWarning: Columns (230,231,237,238) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test2.csv")


,field_1,field_2,field_3,field_4,field_5,field_6,field_7,field_8,field_9,field_10,...,field_348,field_349,field_350,field_351,field_352,field_353,field_354,field_355,field_356,field_357
1471,dish_1550704750,136.949997,83.0,2.988,0.0,25.73,ingr_0000000028,chicken,83.0,136.95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
